<a href="https://colab.research.google.com/github/DenisKai7/invoice_generator/blob/main/fun_run_unipma.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# Install library yang diperlukan
!pip install gspread pandas fpdf2 pyqrcode pypng requests pillow
!pip install --upgrade google-auth-oauthlib google-auth-httplib2

# Import library yang dibutuhkan
import gspread
from google.colab import auth
from google.auth import default
import pandas as pd
from fpdf import FPDF
import os
from google.colab import drive
from datetime import datetime
import requests
from io import BytesIO
from PIL import Image
import urllib.parse
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload

# Autentikasi Google Colab dengan Google Sheets & Drive
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Inisialisasi Google Drive API
drive_service = build('drive', 'v3', credentials=creds)

# Mount Google Drive untuk menyimpan invoice
drive.mount('/content/drive')

# Buka spreadsheet
spreadsheet_url = "https://docs.google.com/spreadsheets/d/1Uu5tAcmE68nM21mBTuuV9HUrtN-9yoEc4o2jz_fPbjE/edit?gid=0#gid=0"
sh = gc.open_by_url(spreadsheet_url)
worksheet = sh.get_worksheet(0)

# Ambil semua data dan periksa kolom yang tersedia
expected_headers = ['KODE_INVOICE', 'NAMA', 'KATEGORI', 'BARCODE', 'NO_WA', 'ALAMAT', 'link_whatsapp', 'template_pesan']
records = worksheet.get_all_records(expected_headers=expected_headers)
df = pd.DataFrame.from_records(records)


# Periksa nama kolom yang sebenarnya
print("Kolom yang tersedia di spreadsheet:")
print(df.columns.tolist())

# Pastikan nama kolom sesuai dengan spreadsheet Anda
COLUMN_MAPPING = {
    'invoice_id': 'KODE_INVOICE',
    'name': 'NAMA',
    'category': 'KATEGORI',
    'barcode': 'BARCODE',
    'whatsapp': 'NO_WA',
    'address': 'ALAMAT'
}

# Path logo di Google Drive (sesuaikan dengan path Anda)
LOGO_DIES_PATH = '/content/drive/MyDrive/logo/diesnatalis 1-01.png'  # Ganti dengan path yang benar
LOGO_UNIPMA_PATH = '/content/drive/MyDrive/logo/Unipma.png'  # Ganti dengan path yang benar

class InvoicePDF(FPDF):
    def __init__(self, participant_data):
        super().__init__(format='A4', orientation='L')  # A4 Landscape
        self.participant_data = participant_data
        self.add_page()

    def header(self):
        # Tambahkan background dengan efek fade
        self._add_faded_background()

        # Tambahkan logo jika file ada
        try:
            # Logo Dies Natalis di kiri - ukuran disesuaikan
            if os.path.exists(LOGO_UNIPMA_PATH):
                self.image(LOGO_UNIPMA_PATH, x=15, y=15, w=30)  # Ukuran lebih besar untuk A4
        except Exception as e:
            print(f"⚠️ Error loading UNIPMA logo: {e}")

        try:
            # Logo UNIPMA di kanan - ukuran disesuaikan
            if os.path.exists(LOGO_DIES_PATH):
                self.image(LOGO_DIES_PATH, x=210, y=0, w=100)  # Ukuran lebih besar untuk A4
        except Exception as e:
            print(f"⚠️ Error loading Dies Natalis logo: {e}")

        # Judul
        self.set_y(25)
        self.set_font('Arial', 'B', 18)
        self.cell(0, 10, 'UNIPMA FUN RUN 5K 2025', 0, 1, 'C')
        self.set_font('Arial', 'I', 12)
        self.cell(0, 7, 'Official Invoice', 0, 1, 'C')
        self.ln(5)

    def _add_faded_background(self):
        # Path ke gambar background (sesuaikan dengan path Anda)
        BG_IMAGE_PATH = '/content/drive/MyDrive/logo/asset diesnat fun run/bg1.jpg'  # Ganti dengan path gambar background Anda

        if os.path.exists(BG_IMAGE_PATH):
            try:
                # Simpan posisi Y saat ini
                current_y = self.get_y()

                # Geser gambar 20px ke kiri (nilai negatif) dan perluas lebar untuk menutupi area yang kosong
                x_offset = -15  # Sesuaikan nilai ini untuk mengatur seberapa jauh ke kiri
                adjusted_width = self.w + abs(x_offset)  # Tambahkan lebar untuk kompensasi pergeseran

                # Tambahkan gambar background dengan opacity rendah
                self.image(BG_IMAGE_PATH, x=x_offset, y=0, w=adjusted_width, h=self.h)

                # Tambahkan layer semi-transparan untuk efek fade
                self.set_fill_color(255, 255, 255, 200)  # Putih dengan alpha 200/255 (~78% opacity)
                self.rect(0, 0, self.w, self.h, style='F')

                # Kembalikan posisi Y
                self.set_y(current_y)
            except Exception as e:
                print(f"⚠️ Error adding background: {e}")
        else:
            print("⚠️ Background image not found, skipping...")

    def footer(self):
        # Thank you message di bagian footer tengah
        self.set_y(-15)
        self.set_font('Arial', 'I', 12)
        self.cell(0, 10, 'System By : Jofanza Denis Aldida & Yoga Gondrong', 0, 0, 'C')

    def create_invoice(self):
        # Informasi invoice
        self.set_font('Arial', 'B', 14)
        self.cell(0, 10, f'INVOICE #{self.participant_data[COLUMN_MAPPING["invoice_id"]]}', 0, 1, 'L')

        # Tanggal
        self.set_font('Arial', '', 11)
        self.cell(0, 8, f'Date: {datetime.now().strftime("%d %B %Y")}', 0, 1, 'L')
        self.ln(8)

        # Garis pemisah
        self.line(10, self.get_y(), 287, self.get_y())
        self.ln(10)

        # Layout 3 kolom dengan alignment yang presisi
        col_width = 85
        start_y = self.get_y()  # Simpan posisi Y awal

        # Kolom 1: Data peserta
        self.set_font('Arial', 'B', 12)
        self.cell(col_width, 8, 'PARTICIPANT DETAILS:', 0, 1, 'L')
        self.set_font('Arial', '', 11)

        # Simpan posisi Y untuk alignment
        y_details = self.get_y()

        # Initialize name_y at the beginning
        name_y = y_details

        # Nama (baseline untuk alignment)
        self.set_font('Arial', 'B', 11)
        self.cell(col_width, 6, 'Nama:', ln=1, align='L')
        self.set_font('Arial', '', 11)
        self.cell(col_width, 6, str(self.participant_data[COLUMN_MAPPING["name"]]), ln=1, align='L')
        self.ln(2)
        name_y = self.get_y()

        # WhatsApp jika ada
        if COLUMN_MAPPING["whatsapp"] in self.participant_data and self.participant_data[COLUMN_MAPPING["whatsapp"]]:
            whatsapp_num = str(self.participant_data[COLUMN_MAPPING["whatsapp"]])
            clean_number = ''.join(filter(str.isdigit, whatsapp_num))
            if clean_number.startswith('0'):
                clean_number = '+62' + clean_number[1:]
            elif not clean_number.startswith('62'):
                clean_number = '+62' + clean_number
            else:
                clean_number = '+' + clean_number

            self.set_font('Arial', 'B', 11)
            self.cell(col_width, 6, 'No WhatsApp:', ln=1, align='L')
            self.set_font('Arial', '', 11)
            self.cell(col_width, 6, clean_number, ln=1, align='L')
            self.ln(2)

        # Alamat jika ada
        if COLUMN_MAPPING["address"] in self.participant_data:
            address = self.participant_data[COLUMN_MAPPING["address"]]
            if pd.notna(address) and str(address).strip() != '':
                self.set_font('Arial', 'B', 11)
                self.cell(col_width, 6, 'Alamat:', ln=1, align='L')
                self.set_font('Arial', '', 11)
                self.multi_cell(col_width, 6, str(address), align='L')
                self.ln(2)

        # Kategori
        self.set_font('Arial', 'B', 11)
        self.cell(col_width, 6, 'Kategori:', ln=1, align='L')
        self.set_font('Arial', '', 11)
        self.cell(col_width, 6, str(self.participant_data[COLUMN_MAPPING["category"]]), ln=1, align='L')
        self.ln(2)

        # Kolom 2: Detail pengambilan race pack (tengah)
        self.set_xy(120, y_details - 5)  # Sejajar dengan "PARTICIPANT DETAILS"
        self.set_font('Arial', 'B', 12)
        self.cell(col_width, 8, 'PENGAMBILAN RACE PACK:', 0, 1, 'L')

        # Posisikan konten race pack sejajar dengan "Name" di X=100
        self.set_xy(100, name_y - 10)  # X=100, Y sejajar dengan "Name"
        self.set_font('Arial', '', 11)

        # Format lengkap dengan indentasi yang tepat
        race_pack_content = [
          ("Tanggal:", 0),                     # Header - no indent
          ("- 30 Mei 2025", 0),                # Date 1 - no indent
          ("- Sesi 1: 08.00-11.00", 10),       # Session 1 - indent 10
          ("- Sesi 2: 13.00-16.00", 10),       # Session 2 - indent 10
          ("- 31 Mei 2025", 0),                # Date 2 - no indent
          ("- Sesi 1: 08.00-11.00", 10),       # Session 1 - indent 10
          ("- Sesi 2: 13.00-15.00", 10),       # Session 2 - indent 10
          ("", 0),                             # Empty line
          ("Lokasi:", 0),                      # Location header - no indent
          ("Laboratorium Terpadu UNIPMA", 0)   # Location - no indent
        ]

        # Cetak dengan indentasi yang konsisten
        for line, indent in race_pack_content:
            self.set_x(120 + indent)
            self.cell(col_width - indent, 6, line, ln=1, align='L')

        # Kolom 3: Barcode/QR code
        if COLUMN_MAPPING["barcode"] in self.participant_data and self.participant_data[COLUMN_MAPPING["barcode"]]:
            try:
                response = requests.get(self.participant_data[COLUMN_MAPPING["barcode"]])
                img = Image.open(BytesIO(response.content))

                temp_img_path = f"/tmp/barcode_{self.participant_data[COLUMN_MAPPING['invoice_id']]}.png"
                img.save(temp_img_path)

                # Posisi di kolom kanan, sejajar dengan "Name"
                x_position = 220
                y_position = name_y - 15  # Adjust to align with "Name"

                self.image(temp_img_path, x=x_position, y=y_position, w=60)

                # Teks di bawah barcode
                self.set_xy(x_position, y_position + 56)
                self.set_font('Arial', 'I', 10)
                self.cell(60, 6, 'Scan this barcode for verification', 0, 1, 'C')

                # Teks di bawah barcode 2
                self.set_xy(x_position, y_position + 61)
                self.set_font('Arial', 'I', 10)
                self.cell(60, 6, 'Thank you for joining our event!', 0, 1, 'C')

                os.remove(temp_img_path)
            except Exception as e:
                print(f"⚠️ Error adding barcode: {e}")

        # Notes di bagian bawah - diposisikan lebih tinggi
        notes_y = max(self.get_y() + 1, 120)  # Mengurangi 15pt dari posisi sebelumnya
        self.set_xy(10, notes_y)
        self.set_font('Arial', 'I', 11)
        notes = [
            "Catatan:",
            "- Harap datang tepat waktu sesuai sesi yang dipilih",
            "- Tunjukkan barcode ini saat pengambilan race pack",
            "- Bawa identitas yang valid (KTP/SIM/KTM/KARTU PELAJAR)"
        ]
        for note in notes:
            self.cell(0, 6, note, ln=1, align='L')

def generate_invoice_pdf(participant_data):
    # Buat PDF
    pdf = InvoicePDF(participant_data)
    pdf.create_invoice()

    # Buat folder penyimpanan di Colab
    local_invoice_dir = "/content/invoices"
    os.makedirs(local_invoice_dir, exist_ok=True)

    # Simpan PDF lokal
    pdf_filename = f"FUN RUN UNIPMA_{participant_data[COLUMN_MAPPING['invoice_id']]}.pdf"
    local_pdf_path = f"{local_invoice_dir}/{pdf_filename}"
    pdf.output(local_pdf_path)

    return local_pdf_path, pdf_filename

def upload_to_drive(file_path, file_name, folder_id=None):
    """Upload file ke Google Drive dan set permissions"""
    file_metadata = {
        'name': file_name,
        'parents': [folder_id] if folder_id else None
    }

    media = MediaFileUpload(file_path, mimetype='application/pdf')

    file = drive_service.files().create(
        body=file_metadata,
        media_body=media,
        fields='id,webViewLink'
    ).execute()

    # Set permissions agar bisa diakses oleh siapa saja dengan link
    permission = {
        'type': 'anyone',
        'role': 'reader'
    }

    drive_service.permissions().create(
        fileId=file['id'],
        body=permission
    ).execute()

    return file['webViewLink']

def generate_whatsapp_link(phone_number):
    if pd.isna(phone_number) or not phone_number:
        return ""

    # Format nomor WhatsApp (62XXXXXXXXXX)
    clean_number = ''.join(filter(str.isdigit, str(phone_number)))
    if clean_number.startswith('0'):
        clean_number = '62' + clean_number[1:]
    elif not clean_number.startswith('62'):
        clean_number = '62' + clean_number
    return f"https://wa.me/{clean_number}"

def generate_message_template(participant_data, drive_link):
    # Format WhatsApp number dengan +62
    whatsapp_num = str(participant_data.get(COLUMN_MAPPING['whatsapp'], ''))
    if whatsapp_num:
        clean_number = ''.join(filter(str.isdigit, whatsapp_num))
        if clean_number.startswith('0'):
            clean_number = '+62' + clean_number[1:]
        elif not clean_number.startswith('62'):
            clean_number = '+62' + clean_number
        else:
            clean_number = '+' + clean_number
    else:
        clean_number = ''

    return f"""
Halo {participant_data[COLUMN_MAPPING['name']]},

Terima kasih telah mendaftar di *UNIPMA FUN RUN 5K 2025*!
Berikut invoice resmi Anda:

🔹 *Kode Invoice:* {participant_data[COLUMN_MAPPING['invoice_id']]}
🔹 *Kategori:* {participant_data[COLUMN_MAPPING['category']]}
🔹 *Alamat:* {participant_data[COLUMN_MAPPING['address']]}


📥 *Download Invoice:*
{drive_link}

*Info Pengambilan Race Pack:*
📅 *Tanggal:*
- 30 Mei 2025: Sesi 1 (08.00-11.00) / Sesi 2 (13.00-16.00)
- 31 Mei 2025: Sesi 1 (08.00-11.00) / Sesi 2 (13.00-15.00)
🏫 *Lokasi:* Laboratorium Terpadu UNIPMA

*Catatan:*
- Harap datang tepat waktu sesuai sesi yang dipilih
- Tunjukkan barcode yang ada pada invoice saat pengambilan racepack
- Bawa identitas yang valid(KTP/SIM/KTM/KARTU PELAJAR)

Mohon simpan invoice ini sebagai bukti pendaftaran.
Jika ada pertanyaan, hubungi kami via WhatsApp.

Salam,
*Panitia Unipma Fun Run 5K 2025*
"""

def process_all_participants():
    updates = []
    errors = []

    # ID folder tujuan di Google Drive (ganti dengan folder ID Anda)
    DRIVE_FOLDER_ID = '1A8ADXVlMOHPJsBhvaxZuBjL7egO3Wq2_'  # Ganti dengan folder ID tujuan

    for index, row in df.iterrows():
        try:
            invoice_id = row[COLUMN_MAPPING['invoice_id']]
            print(f"\nMemproses peserta: {row.get(COLUMN_MAPPING['name'], '')} (Invoice: {invoice_id})")

            # 1. Generate PDF
            local_pdf_path, pdf_filename = generate_invoice_pdf(row)

            # 2. Upload ke Google Drive dan dapatkan link
            drive_link = upload_to_drive(local_pdf_path, pdf_filename, DRIVE_FOLDER_ID)
            print(f"✅ Invoice diupload ke: {drive_link}")

            # 3. Generate link WhatsApp
            whatsapp_link = generate_whatsapp_link(row.get(COLUMN_MAPPING['whatsapp'], ''))
            updates.append((index, 'link_whatsapp', whatsapp_link))

            # 4. Buat template pesan
            message = generate_message_template(row, drive_link)
            updates.append((index, 'template_pesan', message))

            # 5. Bersihkan file lokal
            os.remove(local_pdf_path)

        except Exception as e:
            error_msg = f"❌ Error saat memproses {row.get(COLUMN_MAPPING['name'], '')}: {str(e)}"
            print(error_msg)
            errors.append((index, error_msg))

    # Update spreadsheet
    for update in updates:
        row_idx, col_name, value = update
        try:
            # Cari kolom yang sesuai
            col_names = [col.lower() for col in df.columns]
            target_col = col_name.lower()

            if target_col in col_names:
                col_idx = col_names.index(target_col) + 1  # +1 karena indeks spreadsheet mulai dari 1
                worksheet.update_cell(row_idx + 2, col_idx, value)
                print(f"✔️ Updated {col_name} untuk baris {row_idx + 2}")
            else:
                print(f"⚠️ Kolom {col_name} tidak ditemukan di spreadsheet")
        except Exception as e:
            print(f"❌ Gagal mengupdate spreadsheet untuk baris {row_idx}: {str(e)}")

    # Tampilkan error summary
    if errors:
        print("\n⛔ Error Summary:")
        for error in errors:
            print(error[1])

# Jalankan proses utama
process_all_participants()
print("\n✅ Proses selesai!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Kolom yang tersedia di spreadsheet:
['KODE_INVOICE', 'NAMA', 'KATEGORI', 'BARCODE', 'NO_WA', 'ALAMAT', 'link_whatsapp', 'template_pesan']

Memproses peserta: Yusuf Mansur (Invoice: YUS-1-4058)
⚠️ Error adding background: FPDF.set_fill_color() takes from 2 to 4 positional arguments but 5 were given


<ipython-input-8-66b12096619a>:88: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  self.set_font('Arial', 'B', 18)
<ipython-input-8-66b12096619a>:89: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=1 use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  self.cell(0, 10, 'UNIPMA FUN RUN 5K 2025', 0, 1, 'C')
<ipython-input-8-66b12096619a>:90: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  self.set_font('Arial', 'I', 12)
<ipython-input-8-66b12096619a>:91: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=1 use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  self.cell(0, 7, 'Official Invoice', 0, 1, 'C')
<ipython-input-8-66b12096619a>:129: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  self.set_font('Arial

✅ Invoice diupload ke: https://drive.google.com/file/d/1ZyJIL3s4nFppYBqdBbfEDyzvKtD78mOg/view?usp=drivesdk

Memproses peserta: DEVINA PUTRI HARDIANA  (Invoice: DEV-1-5952)
⚠️ Error adding background: FPDF.set_fill_color() takes from 2 to 4 positional arguments but 5 were given
✅ Invoice diupload ke: https://drive.google.com/file/d/1mo3jNIhoWXgw84b61nNczk2aP0HzDpqF/view?usp=drivesdk

Memproses peserta: TATIK HAYATI (Invoice: TAT-1-4394)
⚠️ Error adding background: FPDF.set_fill_color() takes from 2 to 4 positional arguments but 5 were given
✅ Invoice diupload ke: https://drive.google.com/file/d/1ugLsmnVNtl5dvYL5kguwxEndTNuHwurx/view?usp=drivesdk

Memproses peserta: Zulvan Syarief Imamuddin (Invoice: ZUL-1-7240)
⚠️ Error adding background: FPDF.set_fill_color() takes from 2 to 4 positional arguments but 5 were given
✅ Invoice diupload ke: https://drive.google.com/file/d/1C2zoOGn_qXwZmfsN0FY4U_M_bGENAheO/view?usp=drivesdk

Memproses peserta: ERIL SATRYO SANTOSO (Invoice: ERI-1-6519)
⚠️ Er